In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, learning_curve,cross_val_score
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor


In [60]:
df=pd.read_csv('data/train.csv')

In [61]:
#تبدیل کلید دیکشنری استانداردبه ستون های جدول (دیکشنری ما استاندارد نبود نمیشه ازش استفاده کرد) 
# df = pd.concat([df, df["product_description"].apply(pd.Series)], axis=1)
# df




# df['category']=df.product_description.apply(lambda x:x.split(',')[-2].split(':')[1])

# for i in range(0,(df.shape[0])-1):
#     df['category']=df.product_description[i].split(',')[-2].split(':')[1]
#     df['brand']=df.product_description[i].split(',')[-1].split(':')[1]



# df.product_description[6].split(',')[-1]    #" 'برند': 'متفرقه'}"
# df.product_description[0].split(',')[-1].split(':')    #[" 'برند'", " 'متفرقه'}"]

In [62]:
# Python3 code to demonstrate
# convert dictionary string to dictionary
import ast

def convert_to_dict(string):
    dic = ast.literal_eval(string)
    return dic
df['product_description_dic']=df['product_description'].apply(convert_to_dict)

In [63]:
#برای مقایسه ی اینکه آیا تمامی کلیدهای دیکشنری شبیه به هم یا ترتیب مشابه دارند یا نه که نداشتند:\
# def check_keys_of_column():
#     for i in range(0,df.shape[0]):
#         print(df['product_description_dic'][i].keys())
# check_keys_of_column()        

In [64]:

# category_value_list=[]
# category_dic={}
# for i in range(0,df.shape[0]):
#     key=list(df['product_description_dic'][i].keys())
#     for j in range(len(key)):
#         if key[j]=='دسته بندی':
#             a=list(df['product_description_dic'][i].values())[j]
#             category_value_list.append(a)
# category_dic['category']=category_value_list

        
    
    
    
    
    
    
# brand_value_list=[]
# brand_dic={}
# for i in range(0,df.shape[0]):
#     key=list(df['product_description_dic'][i].keys())
#     for j in range(len(key)):
#         if key[j]=='برند':
#             a=list(df['product_description_dic'][i].values())[j]
#             brand_value_list.append(a)
# brand_dic['brand']=brand_value_list

In [65]:
def split_dict_column():
    feature_list=['دسته بندی','برند']
    dic={}
    for feature_name in feature_list:
        val_list=[]
        for i in range(0,df.shape[0]):
            key=list(df['product_description_dic'][i].keys())
            for j in range(len(key)):
                if key[j]==feature_name:
                    val=list(df['product_description_dic'][i].values())[j]
                    val_list.append(val)
        dic[feature_name]=val_list
    new_df=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in dic.items() ]))
    result = pd.concat([df, new_df], axis=1)
    result.drop(columns=['product_description_dic'],inplace=True)
    result.to_csv('final_df.csv')
    return new_df

split_dict_column()

,دسته بندی,برند
0,کابل کمبو,متفرقه
1,کابل کمبو,ادیمکس
2,کابل کمبو,متفرقه
3,کابل کمبو,متفرقه
4,کابل کمبو,دی-لینک
...,...,...
68835,شارژر تبلت و موبایل,موکسوم
68836,شارژر تبلت و موبایل,ال جی
68837,شارژر تبلت و موبایل,مورف
68838,شارژر تبلت و موبایل,ریمکس


In [66]:
df=pd.read_csv('final_df.csv')

In [67]:
df['brand']=df['برند']
df['category']=df['دسته بندی']


In [68]:
df.drop(columns=['Unnamed: 0','برند','دسته بندی','product_description'],inplace=True)

In [69]:
df.isna().sum()

id          0
price       0
brand       0
category    0
dtype: int64

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68840 entries, 0 to 68839
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        68840 non-null  int64 
 1   price     68840 non-null  int64 
 2   brand     68840 non-null  object
 3   category  68840 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.1+ MB


In [71]:
cat_feature_list=['category','brand']
for i in cat_feature_list:
    df[i] = pd.Categorical(df[i])
    df[i] = df[i].cat.codes
    
    
    
    

In [72]:

values = df.groupby('brand').price.median().sort_values().index
brand_weights = range(1, len(values)+1)
brand_dic = {val: weights for (val, weights) in zip(values, brand_weights)}
df.brand.replace(brand_dic, inplace=True)

In [73]:
values = df.groupby('category').price.mean().sort_values().index
category_weights = range(1, len(values)+1)
category_dic = {val: weights for (val, weights) in zip(values, category_weights)}
df.category.replace(category_dic, inplace=True)

In [74]:
df.groupby('brand').price.mean().sort_values()

brand
1       0.000000e+00
2       1.000000e+04
4       1.140000e+05
3       1.152667e+05
6       1.500000e+05
            ...     
1654    6.002500e+08
1655    6.500000e+08
1657    8.500000e+08
1651    8.588483e+08
1658    9.300000e+08
Name: price, Length: 1658, dtype: float64

In [75]:
df.groupby('category').price.mean().sort_values()

category
1      2.259080e+05
2      5.979960e+05
3      6.094286e+05
4      7.128500e+05
5      8.327044e+05
           ...     
176    3.054659e+08
177    5.663234e+08
178    5.722877e+08
179    7.166853e+08
180    7.259864e+08
Name: price, Length: 180, dtype: float64

In [76]:
a=df.corr()
a['price'].sort_values()

id         -0.222299
category    0.440221
brand       0.445468
price       1.000000
Name: price, dtype: float64

In [77]:
X= df.drop(["price"],axis =1)
y= df["price"]

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=5)

In [79]:
X_train.shape

(48188, 3)

In [80]:
X_test

,id,brand,category
51956,51956,288,2
48769,48769,852,38
21755,21755,288,36
53712,53712,288,2
53583,53583,288,2
...,...,...,...
21894,21894,288,36
4718,4718,981,138
26938,26938,288,111
60956,60956,344,19


In [81]:
y_test.shape

(20652,)

In [82]:

lasso=Lasso()
dt=DecisionTreeRegressor()
knn=KNeighborsRegressor()
poly_pipline = make_pipeline(
    PolynomialFeatures(degree=4, include_bias=False),
    LinearRegression() 
)
models=[poly_pipline,lasso,dt,knn]
for model in models:
    model.fit(X_train, y_train)

In [83]:
cv_results_rms = []
for model in models:
    cv_score = cross_val_score(model, X_train,y_train,scoring="neg_root_mean_squared_error", cv=5)
    cv_results_rms.append(cv_score)
    print("%s: %f " % (model, -1 * cv_score.mean()))

Pipeline(steps=[('polynomialfeatures',
                 PolynomialFeatures(degree=4, include_bias=False)),
                ('linearregression', LinearRegression())]): 45332958.132868 
Lasso(): 57765081.690814 
DecisionTreeRegressor(): 48150967.525043 
KNeighborsRegressor(): 42823236.598349 


In [84]:
df.price.mean()

15931632.70313771

In [86]:
y_dt_pred=dt.predict(X_test) 
print("R^2 decisionTree:",r2_score(y_test, y_dt_pred))


R^2 decisionTree: 0.5846092397504223


In [87]:
y_poly_pred=poly_pipline.predict(X_test) 
print("R^2 poly:",r2_score(y_test, y_poly_pred))

R^2 poly: 0.5246271044359281


In [88]:
y_knn_pred=knn.predict(X_test)
print("R^2 poly:",r2_score(y_test, y_knn_pred))

R^2 poly: 0.6080299118216961


# -------------------------------------------------------------------------------------------------------------------

In [130]:
df_test=pd.read_csv('data/test.csv')

In [131]:
import ast

def convert_to_dict(string):
    dic = ast.literal_eval(string)
    return dic
df_test['product_description_dic']=df_test['product_description'].apply(convert_to_dict)

In [132]:

def split_dict_column():
    feature_list=['دسته بندی','برند']
    dic={}
    for feature_name in feature_list:
        val_list=[]
        for i in range(0,df_test.shape[0]):
            key=list(df_test['product_description_dic'][i].keys())
            for j in range(len(key)):
                if key[j]==feature_name:
                    val=list(df_test['product_description_dic'][i].values())[j]
                    val_list.append(val)
        dic[feature_name]=val_list
    new_df=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in dic.items() ]))
    result = pd.concat([df_test, new_df], axis=1)
    result.drop(columns=['product_description_dic'],inplace=True)
    result.to_csv('final_df_test.csv')
    return new_df

split_dict_column()




,دسته بندی,برند
0,باتری لپ‌تاپ,متفرقه
1,باتری لپ‌تاپ,متفرقه
2,استند و پایه خنک‌کننده,کیو وای اچ
3,باتری لپ‌تاپ,متفرقه
4,باتری لپ‌تاپ,متفرقه
...,...,...
34257,شارژر تبلت و موبایل,باسئوس
34258,شارژر تبلت و موبایل,متفرقه
34259,شارژر تبلت و موبایل,ریمکس
34260,شارژر تبلت و موبایل,متفرقه


In [133]:
df_test=pd.read_csv('final_df_test.csv')
df_test['brand']=df_test['برند']
df_test['category']=df_test['دسته بندی']
df_test.drop(columns=['Unnamed: 0','برند','دسته بندی','product_description'],inplace=True)

In [134]:
cat_feature_list=['category','brand']
for i in cat_feature_list:
    df_test[i] = pd.Categorical(df_test[i])
    df_test[i] = df_test[i].cat.codes
    
    


In [135]:
df_test


,id,brand,category
0,0,819,17
1,1,819,17
2,2,1238,7
3,3,819,17
4,4,819,17
...,...,...,...
34257,34257,300,71
34258,34258,819,71
34259,34259,576,71
34260,34260,819,71


In [136]:
price=knn.predict(df_test) 

In [137]:
price_df=pd.DataFrame(price)

In [138]:
price_df['price']=price_df[0]

In [146]:
result

,id,price
0,0,4088000.0
1,1,4088000.0
2,2,2780000.0
3,3,4088000.0
4,4,4088000.0
...,...,...
34257,34257,312000.0
34258,34258,2009378.0
34259,34259,4617600.0
34260,34260,2009378.0


In [140]:
result = pd.concat([df_test, price_df], axis=1)

In [143]:
result.drop(columns=['brand','category',0],inplace=True)

In [151]:
result.to_csv('output.csv',index=False)

In [152]:
pd.read_csv('output.csv')

,id,price
0,0,4088000.0
1,1,4088000.0
2,2,2780000.0
3,3,4088000.0
4,4,4088000.0
...,...,...
34257,34257,312000.0
34258,34258,2009378.0
34259,34259,4617600.0
34260,34260,2009378.0
